In [2]:
using DataFrames

frame = DataFrame(
  V = [1.320, 1.649, 1.706, 1.783, 1.816, 1.831, 1.860, 1.864, 1.871, 1.891, 1.904, 1.921, 1.945, 1.973, 1.995, 2.026, 2.054, 2.079, 2.101, 2.123, 2.137],
  A = [  0.0,   8.0,  30.0, 153.4, 284.3,   370,   645,   898,  1028,  1553,  2022,  2767,  3945,  5630,  8100, 10070, 13850, 17690, 21730, 25190, 27890],
)

Row,V,A
,Float64,Float64
1,1.32,0.0
2,1.649,8.0
3,1.706,30.0
4,1.783,153.4
5,1.816,284.3
6,1.831,370.0
7,1.86,645.0
8,1.864,898.0
9,1.871,1028.0


In [3]:
x = frame.V
y = frame.A

xerr = (rdg -> 0.009rdg + 0.002).(x)

function dgtA(i)
  if i in 1:5
    0.1
  elseif i in 6:13
    1
  else
    10
  end
end

rdgA(i) = y[i]

yerr = (i -> 0.014rdgA(i) + 3dgtA(i)).(1:21)


21-element Vector{Float64}:
   0.30000000000000004
   0.41200000000000003
   0.72
   2.4476000000000004
   4.280200000000001
   8.18
  12.03
  15.572000000000001
  17.392
  24.742
   ⋮
  58.230000000000004
 108.82000000000001
 143.4
 170.98
 223.9
 277.65999999999997
 334.22
 382.66
 420.46000000000004

In [4]:
using Plots

plot(x, y, label="Plot line")
scatter!(x, y, xerr=xerr, yerr=yerr, markersize=3, label="Point")

xlabel!("U, V")
ylabel!("A, μA")

title!("Plot of Current-Voltage Characteristics")

savefig("current-voltage.png")

"/Users/tikhon/Documents/studies/physics-calculations/lab6/current-voltage.png"

In [5]:
using GLM

Ωx = x[end-5:end]
Ωy = y[end-5:end]

model = lm(@formula(Y ~ X), DataFrame(X=Ωx, Y=Ωy))
b, a = coef(model)

@show a b

μb, μa = stderror(model)

@show μa μb


a = 161474.65784349025
b = -317540.4527000831


μa = 4292.039607792453
μb = 8957.575435839943


8957.575435839943

In [6]:
predict(x) = a*x + b

θx = x[end-7:end]
θy = predict.(θx)

scatter(x, y, xerr=xerr, yerr=yerr, markersize=3, label="Point")
xlabel!("U, V")
ylabel!("A, μA")
title!("Linear Regression Applied")

plot!(θx, θy, label="Regression Line")

savefig("regression.png")

"/Users/tikhon/Documents/studies/physics-calculations/lab6/regression.png"

In [7]:
Ub = -b / a

∑(X) = reduce(+, X)

Y(μs) = (μ -> μ ^ 2).(μs) |> ∑ |> √

μUb = Y([ μb / a, b * μa / a^2 ])

0.07622000375072635

In [8]:
using GLM

function getUb(accuracy)
  sampleX = x[end-accuracy:end]
  sampleY = y[end-accuracy:end]

  model = lm(@formula(Y ~ X), DataFrame(X=sampleX, Y=sampleY))
  b, a = coef(model)

  -b / a
end

e = 1.602176634e-19
c = 299792458
λ = 580e-9
Ub = getUb(5)

@show Ub

h = (e / c) * λ * Ub

Ub = 1.966503332107135


6.09554260315452e-34

In [9]:
μλ = 30e-9

μh = (e / c) * Y([ Ub * μλ, λ * μUb ])

3.9398422532600595e-35

In [10]:
@show round.(digits=2, yerr)

round.(digits = 2, yerr) = [0.3, 0.41, 0.72, 2.45, 4.28, 8.18, 12.03, 15.57, 17.39, 24.74, 31.31, 41.74, 58.23, 108.82, 143.4, 170.98, 223.9, 277.66, 334.22, 382.66, 420.46]


21-element Vector{Float64}:
   0.3
   0.41
   0.72
   2.45
   4.28
   8.18
  12.03
  15.57
  17.39
  24.74
   ⋮
  58.23
 108.82
 143.4
 170.98
 223.9
 277.66
 334.22
 382.66
 420.46

In [11]:
using GLM

function getUb(accuracy)
  sampleX = x[end-accuracy:end]
  sampleY = y[end-accuracy:end]

  model = lm(@formula(Y ~ X), DataFrame(X=sampleX, Y=sampleY))
  b, a = coef(model)

  -b / a
end

e = 1.602176634e-19
c = 299792458
λ = 580e-9

planck(δ) = (e / c) * λ * getUb(δ)


planck (generic function with 1 method)

In [23]:
accuracies = 1:20
constants = planck.(accuracies)

xx = (x -> x + 1).(accuracies)
yy = (h -> round(h * 1e34, digits=2)).(constants)

plot(xx, yy, label="")
scatter!(xx, yy, label="My Values")
plot!(xx, fill(6.626, length(xx)), label="Actual Planck Constant", markersize=10)

xlabel!("δ, Amount of Points")
ylabel!("h, J ⋅ s ⋅ 10^-34")

savefig("custom.png")

"/Users/tikhon/Documents/studies/physics-calculations/lab6/custom.png"